<a href="https://colab.research.google.com/github/samibahig/IFT6390/blob/main/Classifications_de_dessins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from torch.autograd import Variable
import torch.nn.functional as F
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import time
from sklearn.model_selection import train_test_split

In [ ]:
train_ds, test_ds = np.load('/content/train.npz'), np.load('/content/test.npz')

FileNotFoundError: ignored

In [ ]:
train_x = train_ds['arr_0']
train_y = train_ds['arr_1']

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(train_x, train_y, train_size=0.8,test_size=0.2, random_state=35)

In [ ]:
X_test = test_ds['arr_0']

In [ ]:
X_test.shape

(60000, 784)

In [ ]:
# Reshape and normalize
x_train = x_train.reshape(x_train.shape[0], 1, 28, 28).astype('float32')
x_valid = x_valid.reshape(x_valid.shape[0], 1, 28, 28).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')
x_train /= 255.0
x_valid /= 255.0
X_test /=255.0

print("Shape of training dataset ndarray: \n", x_train.shape)
print("Shape of Validating dataset ndarray: \n", x_valid.shape)
print("Shape of Testing dataset ndarray: \n", X_test.shape)

Shape of training dataset ndarray: 
 (1200, 1, 28, 28)
Shape of Validating dataset ndarray: 
 (300, 1, 28, 28)
Shape of Testing dataset ndarray: 
 (60000, 1, 28, 28)


In [ ]:
Test_dataSet = torch.utils.data.TensorDataset(torch.from_numpy(X_test).float().reshape(-1,1,28,28))

In [ ]:
batch_size = 100
Test_dataLoad = torch.utils.data.DataLoader(Test_dataSet, batch_size = batch_size, shuffle = False)

In [ ]:
train_dataset = torch.utils.data.TensorDataset(torch.from_numpy(x_train).float().reshape(-1,1,28,28), torch.from_numpy(y_train))
Valid_dataset = torch.utils.data.TensorDataset(torch.from_numpy(x_valid).float().reshape(-1,1,28,28), torch.from_numpy(y_valid))

In [ ]:
batch_size = 100

train_load = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
Valid_load = torch.utils.data.DataLoader(Valid_dataset, batch_size = batch_size, shuffle = False)

In [ ]:
class CNN(nn.Module):
  def __init__(self):
      super(CNN, self).__init__()
      # same padding Input size = output size
      # Same PAdding = (filtersize-1 / 2) ou (3-1 /2)
      self.cnn1 = nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = 3, stride = 1, padding = 1)
      # the output size of each of the 8 features maps
      # [(input_size - filter_size + 2(padding))/stride + 1] = (28 - 3 + 2)1 + 1 = 28
      self.batchnorm1 = nn.BatchNorm2d(8)
      self.relu = nn.ReLU()
      self.maxpool = nn.MaxPool2d(kernel_size = 2)
      # The output size = 28/2 = 14
      self.cnn2 = nn.Conv2d(in_channels = 8, out_channels = 32, kernel_size = 5, stride = 1, padding = 2)
      # Output Size of each of the 32 feature maps (14 - 5 + 2(2) / 1 + 1) = 14
      self.batchnorm2 = nn.BatchNorm2d(32)
      # Flatten the 32 feature maps: 7*7*32 = 1568
      self.fc1 = nn.Linear(1568, 600)
      self.dropout = nn.Dropout(p = 0.5)
      self.fc2 = nn.Linear(600, 10)
  # Defining the forward pass    
  def forward(self, x):
      out = self.cnn1(x)
      out = self.batchnorm1(out)
      out = self.relu(out)
      out = self.maxpool(out)
      out = self.cnn2(out)
      out = self.batchnorm2(out)
      out = self.relu(out)
      out = self.maxpool(out)
      # Flatten the 32 feature maps from Max Pool to feed it to the FC1 (100, 1568)
      out = out.view(100, 1568)
      #then we forward through our fully connected layer
      out = self.fc1(out)
      out = self.relu(out)
      out = self.dropout(out)
      out = self.fc2(out)
      return out

In [ ]:
model = CNN()
CUDA = torch.cuda.is_available()
if CUDA:
    model = model.cuda()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [ ]:
num_epochs = 20 
train_loss = []
train_accuracy = []
Valid_loss = []
Valid_accuracy = []

for epoch in range(num_epochs):
    correct = 0 
    iterations = 0 
    iter_loss = 0.0
    model.train()

    for i,(inputs, labels) in enumerate(train_load):
        if CUDA:
            inputs = inputs.cuda()
            labels = labels.cuda()
        output = model(inputs)
        loss = loss_fn(output, labels)
        iter_loss += loss.item()


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(output, 1)
        correct += (predicted == labels).sum().item()
        iterations += 1
    train_loss.append(iter_loss/iterations)
    train_accuracy.append(correct / len(train_dataset))
    print(correct / len(train_dataset))

0.18
0.2808333333333333
0.2991666666666667
0.345
0.37083333333333335
0.385
0.4216666666666667
0.435
0.42916666666666664
0.455
0.46
0.4425
0.46416666666666667
0.49333333333333335
0.5058333333333334
0.4791666666666667
0.4683333333333333
0.49833333333333335
0.5283333333333333
0.5566666666666666


In [ ]:
print(train_loss)

[12.728054722150167, 1.857569009065628, 1.6607242623964946, 1.5752504169940948, 1.529810110727946, 1.4954232374827068, 1.4421628415584564, 1.43203204870224, 1.4274177451928456, 1.3791746000448863, 1.36418550213178, 1.3310560484727223, 1.3427033424377441, 1.2734635670979817, 1.2595767776171367, 1.3034159044424694, 1.302490621805191, 1.2413116991519928, 1.225953181584676, 1.1774118145306904]


In [ ]:
for epoch in range(num_epochs):
    valid_loss = []
    ep_loss = 0
    correct = 0 
    iterations = 0 
    model.eval()

    for i,(inputs, labels) in enumerate(Valid_load):
        if CUDA:
            inputs = inputs.cuda()
            labels = labels.cuda()
        output = model(inputs)
        loss = loss_fn(output, labels)
        iter_loss += loss.item()


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(output, 1)
        correct += (predicted == labels).sum().item()
        iterations += 1
    valid_loss.append(ep_loss/iterations)
    Valid_accuracy.append((100 * correct / len(Valid_dataset)))


In [ ]:
print(valid_loss)
print(Valid_accuracy)

[0.0]
[49.666666666666664, 51.0, 55.333333333333336, 60.666666666666664, 64.33333333333333, 67.0, 71.33333333333333, 75.33333333333333, 78.66666666666667, 83.33333333333333, 86.33333333333333, 89.66666666666667, 90.33333333333333, 92.66666666666667, 92.0, 94.0, 87.66666666666667, 96.0, 98.0, 99.0]


In [ ]:
for epoch in range(num_epochs):
    Test_loss = []
    ep_loss = 0
    correct = 0 
    iterations = 0 
    model.eval()

    for i,(inputs) in enumerate(Test_dataLoad):
       # if CUDA:
        #    inputs = inputs.cuda()
        output = model(inputs)
       # loss = loss_fn(output)
        #iter_loss += loss.item()


       # optimizer.zero_grad()
        #loss.backward()
        #optimizer.step()

        _, predicted = torch.max(output, 1)
       # correct += (predicted == labels).sum().item()
       # iterations += 1

TypeError: ignored

In [ ]:
 print("Training Loss: {:.3f}, Testing Loss: {:.3f}, Training Accuracy{:.3f}, Testing Accuracy: {:.3f}".format(train_loss[-1], test_loss[-1], train_accuracy[-1], test_accuracy[-1]))

NameError: ignored

In [ ]:
type(test_loss)

NameError: ignored

In [ ]:
device = torch.device('cuda:0')  
X_test = torch.from_numpy(X_test).float().to(device)

In [ ]:
output = model(X_test)
_, pred = torch.max(output)
pd.DataFrame(pred).to_csv('CNN')


RuntimeError: ignored

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
device = torch.device('cuda')

In [ ]:
a = torch.rand(3, 3)
a

tensor([[0.3894, 0.6145, 0.0211],
        [0.6368, 0.0192, 0.7613],
        [0.9107, 0.5537, 0.8451]])

In [ ]:
b = torch.rand(3, 3).to(device)
b

tensor([[0.7042, 0.9249, 0.1200],
        [0.2182, 0.5484, 0.9837],
        [0.7345, 0.5928, 0.4833]], device='cuda:0')

In [ ]:
c = torch.rand(3, 3).cuda()
c

tensor([[0.0721, 0.1413, 0.8733],
        [0.3277, 0.5107, 0.6627],
        [0.0875, 0.8770, 0.0057]], device='cuda:0')